In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import re
from pprint import pprint
import country_converter as coco
import datetime
import pycountry

In [2]:
countries = 'https://api.covid19api.com/countries'

In [3]:
response_countries = requests.get(countries).json()
pprint(response_countries)

[{'Country': 'Belgium', 'ISO2': 'BE', 'Slug': 'belgium'},
 {'Country': 'Mexico', 'ISO2': 'MX', 'Slug': 'mexico'},
 {'Country': 'Israel', 'ISO2': 'IL', 'Slug': 'israel'},
 {'Country': 'Algeria', 'ISO2': 'DZ', 'Slug': 'algeria'},
 {'Country': 'Bosnia and Herzegovina',
  'ISO2': 'BA',
  'Slug': 'bosnia-and-herzegovina'},
 {'Country': 'Luxembourg', 'ISO2': 'LU', 'Slug': 'luxembourg'},
 {'Country': 'Samoa', 'ISO2': 'WS', 'Slug': 'samoa'},
 {'Country': 'Belize', 'ISO2': 'BZ', 'Slug': 'belize'},
 {'Country': 'Benin', 'ISO2': 'BJ', 'Slug': 'benin'},
 {'Country': 'Croatia', 'ISO2': 'HR', 'Slug': 'croatia'},
 {'Country': 'Lebanon', 'ISO2': 'LB', 'Slug': 'lebanon'},
 {'Country': 'Russian Federation', 'ISO2': 'RU', 'Slug': 'russia'},
 {'Country': 'South Africa', 'ISO2': 'ZA', 'Slug': 'south-africa'},
 {'Country': 'South Georgia and the South Sandwich Islands',
  'ISO2': 'GS',
  'Slug': 'south-georgia-and-the-south-sandwich-islands'},
 {'Country': 'Anguilla', 'ISO2': 'AI', 'Slug': 'anguilla'},
 {'C

In [ ]:
len(response_countries)

In [ ]:
name = []
slug = []
ISO2 = []
for i in range(len(response_countries)):
    name.append(response_countries[i]["Country"])
    slug.append(response_countries[i]["Slug"])
    ISO2.append(response_countries[i]["ISO2"])

countries_slug_df = pd.DataFrame({"Entity": name,
                                 "slug": slug,
                                 "ISO2": ISO2})
countries_slug_df.head()

In [ ]:
countries_slug_df.sort_values(by = ["Entity"], inplace = True, ignore_index = True)
countries_slug_df.head()

In [ ]:
countries_slug_df.to_csv("countries_raw.csv")

In [15]:
countries_slug_df = pd.read_csv("countries_raw.csv")
countries_slug_df = countries_slug_df[["Entity", "slug", "ISO2"]]
countries_slug_df.head()

,Entity,slug,ISO2
0,ALA Aland Islands,ala-aland-islands,AX
1,Afghanistan,afghanistan,AF
2,Albania,albania,AL
3,Algeria,algeria,DZ
4,American Samoa,american-samoa,AS


In [ ]:
len(countries_slug_df)

In [16]:
countries_slug_df.describe()

,Entity,slug,ISO2
count,248,248,247
unique,248,248,247
top,Dominica,tuvalu,TN
freq,1,1,1


In [ ]:
entity1= list(set(countries_slug_df["Entity"]))
standard_names = coco.convert(names=entity1, to='name_short')
standardized_names1_df = pd.DataFrame({"Entity" : entity1,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names1_df.head()

In [ ]:
std_countries_slug_df = pd.merge(countries_slug_df, standardized_names1_df, how = "left", left_on='Entity', right_on='Entity')
std_countries_slug_df.sort_values(by = "Entity", inplace = True)
std_countries_slug_df.head()

In [ ]:
std_countries_slug_df.to_csv("countries_slug.csv")

store response_countries in to a df.
Import csv into a df.
parse the country name column to add hyphens replacing spaces.
do a lookup on response_countries df to get the slugs.
create for loop to check all the slugs and make separate api calls.

In [ ]:
stringency_df = pd.read_csv("covid-stringency-index.csv")
stringency_df.head()

In [ ]:
len(stringency_df)

In [ ]:
stringency_df["Entity"].nunique()

In [ ]:
stringency_df['month'] = pd.DatetimeIndex(stringency_df['Date']).month
stringency_df['year'] = pd.DatetimeIndex(stringency_df['Date']).year
stringency_df.head()

In [ ]:
stringency_df.loc[stringency_df['Entity'] == "Timor", 'Entity'] = "Timor-leste"

In [ ]:
stringency_df.describe()

In [ ]:
stringency_df.loc[stringency_df['Entity'] == "Timor-leste"]

In [ ]:
entity2= list(set(stringency_df["Entity"]))
standard_names2 = coco.convert(names=entity2, to='name_short')
standardized_names2_df = pd.DataFrame({"Entity" : entity2,
                                      "standard_names": standard_names2}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names2_df.head(10)

In [ ]:
cleaned_stringency_df = stringency_df.groupby(by = ["Entity", "Code", "month", "year"], as_index = False)["stringency_index"].mean()
cleaned_stringency_df.head(-10)

In [ ]:
std_stringency_df = pd.merge(cleaned_stringency_df, standardized_names2_df, how = "left", left_on='Entity', right_on='Entity')
std_stringency_df.sort_values(by = "Entity", inplace = True)
std_stringency_df.head()

In [ ]:
std_stringency_df.to_csv("cleaned_stringency.csv")

In [ ]:
merged_stringency_df =pd.merge(std_countries_slug_df, std_stringency_df, how = "inner", on = "standard_names")
merged_stringency_df = merged_stringency_df[merged_stringency_df["standard_names"] != "not found"]
merged_stringency_df.rename(columns = {"Entity_x": "Entity", "Code" : "ISO3"}, inplace=True)
merged_stringency_df = merged_stringency_df[["Entity" , "standard_names", "slug", "ISO2", "ISO3", "month", "year", "stringency_index"]]
merged_stringency_df.sort_values( by = ["standard_names", "year", "month"], inplace = True, ignore_index=True)
merged_stringency_df.head()

In [ ]:
merged_stringency_df.to_csv("merged_stringency.csv")

In [4]:
merged_stringency_df = pd.read_csv("merged_stringency.csv")

In [5]:
merged_stringency_df["standard_names"].nunique()

181

In [ ]:
std_countries_slug_df[["Entity"]][~std_countries_slug_df["standard_names"].isin(set(merged_stringency_df["standard_names"]))].drop_duplicates().reset_index(drop = True)

In [ ]:
std_stringency_df[["Entity"]][~std_stringency_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

In [ ]:
merged_stringency_df= pd.read_csv("merged_stringency.csv")

In [ ]:
country_names= list(set(merged_stringency_df['slug']))
country_names.sort()
country_names

In [ ]:
len(country_names)

In [ ]:
country_std_names= list(set(merged_stringency_df['standard_names']))
country_std_names.sort()
country_std_names

In [ ]:
# Target country
# target_country = "India"
lat = []
lng = []
contry_geo_name = []
# Build the endpoint URL
for country in range(len(country_std_names)):
    geo_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(country_std_names[country], gkey)
    geo_data = requests.get(geo_url).json()
    contry_geo_name.append(geo_data["results"][0]["address_components"][0]["long_name"])
    lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
    time.sleep(1)
# pprint(geo_data)

In [ ]:
contry_coord = pd.DataFrame({"Entity": contry_geo_name,
                            "lat": lat,
                            "lng" : lng,
                            "standard_names" : country_std_names})
contry_coord.head()

In [ ]:
contry_coord.to_csv("contry_coord.csv")

In [ ]:
contry_coord = pd.read_csv("contry_coord.csv")

In [ ]:
contry_coord["Entity"].nunique()

In [ ]:
base_url = 'https://api.covid19api.com/total/country/'
# https://api.covid19api.com/total/country/india/status/confirmed
entity = []
confirmed = []
deaths = []
recovered = []
active = []
date = []

In [ ]:
# test_url = "https://api.covid19api.com/total/country/albania"
# covid_response = requests.get(test_url).json()
# print(len(covid_response))

In [ ]:
for country in range(len(country_names)):
    search_url = base_url + country_names[country]
    covid_response = requests.get(search_url).json()
#     print(len(covid_response))
    for i in range(len(covid_response)):
        entity.append(covid_response[i]["Country"])
        confirmed.append(covid_response[i]["Confirmed"])
        deaths.append(covid_response[i]["Deaths"])
        recovered.append(covid_response[i]["Recovered"])
        active.append(covid_response[i]["Active"])
        date.append(covid_response[i]["Date"])
    time.sleep(1)

In [ ]:
global_covid_df = pd.DataFrame({"Entity" : entity,
                               "Confirmed" : confirmed,
                               "Deaths" : deaths,
                               "Recovered" : recovered,
                               "Active" : active,
                               "Date" : date})
global_covid_df.sort_values(by = ["Entity", "Date"], inplace = True, ignore_index= True)
global_covid_df.head(-10)

In [ ]:
search_url = "https://api.covid19api.com/total/country/india"
covid_response = requests.get(search_url).json()
covid_response[0]["Date"]

In [ ]:
global_covid_df.to_csv("global_covid_stats.csv")

In [6]:
global_covid_df = pd.read_csv("global_covid_stats.csv")

global_covid_df.head()

,Unnamed: 0,Entity,Confirmed,Deaths,Recovered,Active,Date
0,0,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
1,1,Afghanistan,0,0,0,0,2020-01-23T00:00:00Z
2,2,Afghanistan,0,0,0,0,2020-01-24T00:00:00Z
3,3,Afghanistan,0,0,0,0,2020-01-25T00:00:00Z
4,4,Afghanistan,0,0,0,0,2020-01-26T00:00:00Z


In [7]:
global_covid_df["Confirmed_daily"] =global_covid_df["Confirmed"].diff().fillna(0)
global_covid_df["Deaths_daily"] =global_covid_df["Deaths"].diff().fillna(0)
global_covid_df["Recovered_daily"] =global_covid_df["Recovered"].diff().fillna(0)
global_covid_df["Active_daily"] =global_covid_df["Active"].diff().fillna(0)

# num = global_covid_df._get_numeric_data()
# num[num < 0] = 0
global_covid_df.loc[global_covid_df['Confirmed_daily'] < 0 , 'Confirmed_daily'] = global_covid_df["Confirmed"]
global_covid_df.loc[global_covid_df['Deaths_daily'] < 0 , 'Deaths_daily'] = global_covid_df["Deaths"]
global_covid_df.loc[global_covid_df['Recovered_daily'] < 0 , 'Recovered_daily'] = global_covid_df["Recovered"]
global_covid_df.loc[global_covid_df['Active_daily'] < 0 , 'Active_daily'] = global_covid_df["Active"]
global_covid_df = global_covid_df[['Entity','Date', 'Confirmed_daily', 'Deaths_daily', 'Recovered_daily',
       'Active_daily']]
global_covid_df.head(-10)

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
57044,Zimbabwe,2020-12-04T00:00:00Z,123.0,1.0,48.0,74.0
57045,Zimbabwe,2020-12-05T00:00:00Z,70.0,10.0,42.0,18.0
57046,Zimbabwe,2020-12-06T00:00:00Z,101.0,0.0,36.0,65.0
57047,Zimbabwe,2020-12-07T00:00:00Z,121.0,3.0,92.0,26.0


In [ ]:
global_covid_df["Entity"].nunique()

In [ ]:
len(global_covid_df)

In [ ]:
global_covid_df.to_csv("global_covid_daily.csv")

In [8]:
global_covid_df['month'] = pd.DatetimeIndex(global_covid_df['Date']).month
global_covid_df['year'] = pd.DatetimeIndex(global_covid_df['Date']).year
global_covid_df.head()

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,month,year
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0,1,2020


In [9]:
global_covid_monthly_df = global_covid_df.groupby(["Entity", "month", "year"], as_index = False).agg({"Confirmed_daily": "sum",
                                                                                   "Deaths_daily": "sum",
                                                                                   "Recovered_daily": "sum",
                                                                                   "Active_daily" : "sum"})
global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0
...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,6702.0
2049,Zambia,11,2020,1215.0,8.0,1318.0,7226.0
2050,Zambia,12,2020,928.0,16.0,731.0,1781.0
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0


In [10]:
len(global_covid_monthly_df)

2063

In [3]:
std_global_covid_monthly_df = pd.merge(global_covid_monthly_df,standardized_names1_df, how = "inner", on = "Entity")
std_global_covid_monthly_df.head(-10)

NameError: name 'global_covid_monthly_df' is not defined

In [ ]:
std_global_covid_monthly_df.loc[std_global_covid_monthly_df["standard_names"] == "Macau"]

In [ ]:
std_global_covid_monthly_df["standard_names"].nunique()

In [ ]:
std_global_covid_monthly_stringency_df = pd.merge(std_global_covid_monthly_df, merged_stringency_df , 
                                                  how = "inner", on = ["standard_names", "month", "year"])
std_global_covid_monthly_stringency_df = std_global_covid_monthly_stringency_df[['standard_names', 'month', 'year',
        'Confirmed_daily', 'Deaths_daily', 'Recovered_daily', 'Active_daily', 'stringency_index']]
std_global_covid_monthly_stringency_df.head(-10)

In [ ]:
std_global_covid_monthly_stringency_df["standard_names"].nunique()

In [ ]:
main_df = pd.merge(std_global_covid_monthly_stringency_df, contry_coord, how = "left", on = "standard_names")
main_df = main_df[['standard_names', 'month', 'year', 'Confirmed_daily', 'Deaths_daily',
       'Recovered_daily', 'Active_daily', 'stringency_index', 'lat','lng']]
main_df.rename(columns = {'Confirmed_daily' : "Confirmed", 'Deaths_daily' : "Deaths",
                           'Recovered_daily': "Recovered", 'Active_daily' : "Active"}, inplace = True)
main_df.head(-10)

In [ ]:
main_df["standard_names"].nunique()

In [ ]:
main_df.loc[~main_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)

In [ ]:
main_df.loc[~main_df["standard_names"].isin(list(set(contry_coord["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)

In [ ]:
main_df.to_csv("main.csv")

In [ ]:
main_stats_df = main_df.groupby("standard_names", as_index = False).agg({"Confirmed": "sum",
                                                                                   "Deaths": "sum",
                                                                                   "Recovered": "sum",
                                                                                   "Active" : "sum"})
main_stats_df.sort_values('Confirmed', inplace= True, ascending=False, ignore_index=True)

main_stats_df.head(10)

In [ ]:
all_zero_df = main_stats_df.loc[main_stats_df["Confirmed"] == 0]
all_zero_df

In [ ]:
main_df_dropped = main_df.loc[~main_df["standard_names"].isin(list(set(all_zero_df["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)
main_df_dropped["standard_names"].nunique()

In [ ]:
main_stats_df.sort_values('Confirmed', ascending=True, ignore_index=True)

In [ ]:
main_stats_df.to_csv("main_stats_2020.csv")

In [3]:
popluation_2020 = pd.read_csv("population_by_country_2020.csv")
popluation_2020.head(-10)

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
220,Caribbean Netherlands,26265,0.94 %,244,80,328,NaN,N.A.,N.A.,75 %,0.00 %
221,Palau,18109,0.48 %,86,39,460,NaN,N.A.,N.A.,N.A.,0.00 %
222,Cook Islands,17567,0.09 %,16,73,240,NaN,N.A.,N.A.,75 %,0.00 %
223,Anguilla,15026,0.90 %,134,167,90,NaN,N.A.,N.A.,N.A.,0.00 %


## Get 10 countries around mean, 10 countries with least numbers also

In [ ]:
popluation_2020["Country (or dependency)"].nunique()

In [ ]:
popluation_2020[["Country (or dependency)"]][~popluation_2020["Country (or dependency)"].isin(list(set(main_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

In [ ]:
entity3= list(set(popluation_2020["Country (or dependency)"]))
standard_names3 = coco.convert(names=entity3, to='name_short')
standardized_names3_df = pd.DataFrame({"Country (or dependency)" : entity3,
                                      "standard_names": standard_names3}).sort_values(by = "Country (or dependency)").reset_index(drop = True)
standardized_names3_df.head(10)

In [ ]:
std_population_df = pd.merge(popluation_2020, standardized_names3_df, how = "left", on = "Country (or dependency)")
std_population_df = std_population_df[["standard_names", "Population (2020)"]].dropna()
std_population_df.head()

In [ ]:
std_population_df["standard_names"].nunique()

In [ ]:
ultimate_df = pd.merge(main_df_dropped, std_population_df, how = "inner", on = "standard_names")
ultimate_df["Infection_rate"] = ((ultimate_df["Confirmed"]/ ultimate_df["Population (2020)"]*100).map("{:.4f}%".format))
ultimate_df = ultimate_df[["standard_names", "month", "year", "Confirmed", "stringency_index"
                          , "lat", "lng", "Population (2020)", "Infection_rate" ]]
ultimate_df.head()

In [ ]:
ultimate_df["standard_names"].nunique()

In [ ]:
main_df[["standard_names"]][~main_df["standard_names"].isin(list(set(ultimate_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

In [ ]:
ultimate_df.head(20)

In [ ]:
ultimate_df.to_csv("ultimate.csv")

In [4]:
ultimate_df = pd.read_csv("ultimate.csv")
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate
0,Afghanistan,1,2020,0,0.000000,33.93911,67.709953,39074280,0.00%
1,Afghanistan,2,2020,1,2.010690,33.93911,67.709953,39074280,0.00%
2,Afghanistan,3,2020,174,40.831935,33.93911,67.709953,39074280,0.00%
3,Afghanistan,4,2020,1952,80.740000,33.93911,67.709953,39074280,0.01%
4,Afghanistan,5,2020,13081,84.260000,33.93911,67.709953,39074280,0.03%


In [5]:
country_name = []
ISO_2 = []
ISO_3 = []
for country in pycountry.countries:
    country_name.append(country.name)
    ISO_2.append(country.alpha_2)
    ISO_3.append(country.alpha_3)
    
country_ISO_df = pd.DataFrame({"Entity" : country_name,
                               "ISO_2" : ISO_2,
                               "ISO_3" : ISO_3})
country_ISO_df.head()

,Entity,ISO_2,ISO_3
0,Aruba,AW,ABW
1,Afghanistan,AF,AFG
2,Angola,AO,AGO
3,Anguilla,AI,AIA
4,Åland Islands,AX,ALA


In [6]:
entity3= list(set(country_ISO_df["Entity"]))
standard_names = coco.convert(names=entity3, to='name_short')
standardized_names4_df = pd.DataFrame({"Entity" : entity3,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names4_df.head()

,Entity,standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,American Samoa,American Samoa
4,Andorra,Andorra


In [7]:
country_ISO_df = pd.merge(country_ISO_df, standardized_names4_df, how = "inner", on = "Entity")
country_ISO_df = country_ISO_df[["standard_names", "ISO_2", "ISO_3"]]
country_ISO_df.head()

,standard_names,ISO_2,ISO_3
0,Aruba,AW,ABW
1,Afghanistan,AF,AFG
2,Angola,AO,AGO
3,Anguilla,AI,AIA
4,Aland Islands,AX,ALA


In [8]:
country_ISO_df["standard_names"].nunique()

249

In [9]:
ultimate_df = pd.merge(ultimate_df, country_ISO_df, how = "inner" , on = "standard_names")
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,Afghanistan,1,2020,0,0.000000,33.93911,67.709953,39074280,0.00%,AF,AFG
1,Afghanistan,2,2020,1,2.010690,33.93911,67.709953,39074280,0.00%,AF,AFG
2,Afghanistan,3,2020,174,40.831935,33.93911,67.709953,39074280,0.00%,AF,AFG
3,Afghanistan,4,2020,1952,80.740000,33.93911,67.709953,39074280,0.01%,AF,AFG
4,Afghanistan,5,2020,13081,84.260000,33.93911,67.709953,39074280,0.03%,AF,AFG


In [10]:
ultimate_df["standard_names"].nunique()

170

In [11]:
ultimate_df.to_csv("ultimate.csv", index = False)

In [37]:
ultimate_nan_df = ultimate_df[ultimate_df["stringency_index"].isnull()]
ultimate_nan_df

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3


In [38]:
ultimate_nan_df.to_csv("ultimate_nan.csv", index=False)

In [12]:
ultimate_no_null_df = ultimate_df.dropna()
ultimate_no_null_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,Afghanistan,1,2020,0,0.000000,33.93911,67.709953,39074280,0.00%,AF,AFG
1,Afghanistan,2,2020,1,2.010690,33.93911,67.709953,39074280,0.00%,AF,AFG
2,Afghanistan,3,2020,174,40.831935,33.93911,67.709953,39074280,0.00%,AF,AFG
3,Afghanistan,4,2020,1952,80.740000,33.93911,67.709953,39074280,0.01%,AF,AFG
4,Afghanistan,5,2020,13081,84.260000,33.93911,67.709953,39074280,0.03%,AF,AFG


In [13]:
ultimate_no_null_df.to_csv("ultimate_no_null.csv" , index=False)

In [14]:
ultimate_df["standard_names"].nunique()

170

In [15]:
ultimate_no_null_df["standard_names"].nunique()

170

In [16]:
ultimate_no_null_df["Infection_rate_f"] = ultimate_no_null_df["Infection_rate"].str.rstrip('%').astype('float')

In [17]:
covid_stats_df = ultimate_no_null_df.groupby(["standard_names", "ISO_2", "ISO_3", "Population (2020)", "lat", "lng"], as_index= False).agg({"Confirmed" : "sum",
                                                                                      "stringency_index": "mean",
                                                                                      "Infection_rate_f" : "sum"})
covid_stats_df.to_csv("covid_stats_2020.csv", index=False)
covid_stats_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11
1,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,94371,59.981444,0.22
3,Andorra,AD,AND,77287,42.506285,1.521801,7519,40.286864,9.72
4,Angola,AO,AGO,33032075,-11.202692,17.873887,16562,57.541104,0.04


In [4]:
covid_stats_df =  pd.read_csv("covid_stats_2020.csv")
covid_stats_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11
1,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,94371,59.981444,0.22
3,Andorra,AD,AND,77287,42.506285,1.521801,7519,40.286864,9.72
4,Angola,AO,AGO,33032075,-11.202692,17.873887,16562,57.541104,0.04


In [13]:
covid_stats_sorted_df = covid_stats_df.sort_values(by = "Confirmed", ascending = False, ignore_index= True)
covid_stats_sorted_df.to_csv("covid_stats_sorted.csv", index= False)
covid_stats_sorted_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,17459296,56.045880,5.27
1,India,IN,IND,1382345085,20.593684,78.962880,9462809,62.869497,0.68
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,7162978,56.761687,3.37
3,China,CN,CHN,1440297825,35.861660,104.195397,6665209,68.391841,0.46
4,France,FR,FRA,65298930,46.227638,2.213749,5352449,55.223795,8.21


In [6]:
covid_stats_sorted_df_top = covid_stats_sorted_df.head(10)
covid_stats_sorted_df_top

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,17459296,56.045880,5.27
1,India,IN,IND,1382345085,20.593684,78.962880,9462809,62.869497,0.68
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,7162978,56.761687,3.37
3,China,CN,CHN,1440297825,35.861660,104.195397,6665209,68.391841,0.46
4,France,FR,FRA,65298930,46.227638,2.213749,5352449,55.223795,8.21
5,Russia,RU,RUS,145945524,61.524010,105.318756,2764843,50.600998,1.91
6,Spain,ES,ESP,46757980,40.463667,-3.749220,2246032,55.975557,4.81
7,United Kingdom,GB,GBR,67948282,55.378051,-3.435973,2173896,56.888553,3.20
8,Italy,IT,ITA,60446035,41.871940,12.567380,2159937,58.364239,3.58
9,Turkey,TR,TUR,84495243,38.963745,35.243322,1982090,53.585087,2.36


In [7]:
covid_stats_sorted_df_bottom = covid_stats_sorted_df.tail(10)
covid_stats_sorted_df_bottom

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
160,Barbados,BB,BRB,287437,13.193887,-59.543198,307,45.059213,0.10
161,Seychelles,SC,SYC,98453,-4.679574,55.491977,202,33.860586,0.20
162,Monaco,MC,MCO,39290,43.738418,7.424616,154,54.186199,0.39
163,Brunei Darussalam,BN,BRN,438202,4.535277,114.727669,152,38.282705,0.03
164,Dominica,DM,DMA,72017,15.414999,-61.370976,88,35.822549,0.12
165,Fiji,FJ,FJI,897573,-17.713371,178.065032,46,51.240738,0.00
166,Laos,LA,LAO,7293542,19.856270,102.495496,41,34.867858,0.00
167,Timor-Leste,TL,TLS,1322667,-8.874217,125.727539,31,31.385059,0.00
168,Solomon Islands,SB,SLB,689671,-9.645710,160.156194,17,31.102298,0.00
169,Vanuatu,VU,VUT,308337,-15.376706,166.959158,1,38.589624,0.00


In [8]:
def get_median_index(d):
    ranks = d.rank(pct=True)
    close_to_median = abs(ranks - 0.5)
    return close_to_median.idxmin()
median_index = list(covid_stats_sorted_df[["Confirmed"]].apply(get_median_index, 0))
# type(median_index)
median_index[0]

85

In [9]:
covid_stats_sorted_df_mid = covid_stats_sorted_df.iloc[ median_index[0]-5:median_index[0]+5]
covid_stats_sorted_df_mid

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
80,Nigeria,NG,NGA,206984347,9.081999,8.675277,77013,53.885979,0.03
81,Uzbekistan,UZ,UZB,33551824,41.377491,64.585262,75675,50.968375,0.22
82,Malaysia,MY,MYS,32436963,4.210484,101.975766,65697,52.726719,0.21
83,Singapore,SG,SGP,5858322,1.352083,103.819836,58386,49.634418,0.98
84,Ghana,GH,GHA,31181428,7.946527,-1.023194,53653,41.623758,0.18
85,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
86,Luxembourg,LU,LUX,627704,49.815273,6.129583,51656,43.160278,8.22
87,South Korea,KR,KOR,51276977,35.907757,127.766922,48570,47.322044,0.10
88,Finland,FI,FIN,5542237,61.924110,25.748151,47179,36.915609,0.85
89,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11


In [10]:
sample = covid_stats_sorted_df_top.append(covid_stats_sorted_df_mid, ignore_index = True)
sample = sample.append(covid_stats_sorted_df_bottom, ignore_index = True)
sample

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,17459296,56.045880,5.27
1,India,IN,IND,1382345085,20.593684,78.962880,9462809,62.869497,0.68
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,7162978,56.761687,3.37
3,China,CN,CHN,1440297825,35.861660,104.195397,6665209,68.391841,0.46
4,France,FR,FRA,65298930,46.227638,2.213749,5352449,55.223795,8.21
5,Russia,RU,RUS,145945524,61.524010,105.318756,2764843,50.600998,1.91
6,Spain,ES,ESP,46757980,40.463667,-3.749220,2246032,55.975557,4.81
7,United Kingdom,GB,GBR,67948282,55.378051,-3.435973,2173896,56.888553,3.20
8,Italy,IT,ITA,60446035,41.871940,12.567380,2159937,58.364239,3.58
9,Turkey,TR,TUR,84495243,38.963745,35.243322,1982090,53.585087,2.36


In [12]:
sample.to_csv("samples.csv", index= False)